In [130]:
"""
Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции. 
По образу практики, попробуйте создать искусственный датасет с лишними столбцами. 
Целевую метку, при правильной обработке данных, формируйте таким образом, 
чтобы без затруднений её смогла описать линейная модель. Ориентируйтесь на то, что было показано во время занятия, 
и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. 
Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы. 
Чем больше вы фантазируете, тем лучше :) 
Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных
"""

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [131]:
# Создаём сэмпл
n_samples = 890

# числа в диапазоне до 3 в количестве n_samples (т.е. 890 штук)+2 к каждому числу:
number_of_passengers = np.random.choice(3, n_samples) + 2 
price_of_ticket = np.random.choice(35000, n_samples) + 28
invoice_amount = ((np.random.choice(12, n_samples)) * price_of_ticket) + 1538
age_of_passengers = np.random.choice(78, n_samples) + 5

data = pd.DataFrame({'Number_of_passengers': number_of_passengers, 'Age': age_of_passengers, 'Price': price_of_ticket, 'Amount': invoice_amount}) 
data.head(10)

,Number_of_passengers,Age,Price,Amount
0,2,61,22601,137144
1,3,25,21077,191231
2,4,28,26904,136058
3,4,72,9861,40982
4,2,15,3471,36248
5,4,45,10989,100439
6,2,77,11834,37040
7,2,51,1816,21514
8,4,28,34701,348548
9,3,38,29118,59774


In [132]:
X = data[['Number_of_passengers', 'Age', 'Price']]
y = data['Amount']

In [133]:
from sklearn.model_selection import train_test_split
model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.3, 
                                                        random_state=42 )
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [134]:
from sklearn.metrics import mean_absolute_error

print(f'Веса признаков: {model.coef_}') # веса каждого элемента (признака) из переменной Х, определяемые коэффициентами reg.coef_
print(f'Bias: {model.intercept_}') # Смещение: intercept_ float или массив формы (n_targets,)
                                         # Независимый член в линейной модели. Установите 0,0, если .fit_intercept = False

pred_values = model.predict(data[['Number_of_passengers', 'Age', 'Price']]) # фактически это - reg.predict(X)
print(f'Абсолютная ошибка предсказания: {round(mean_absolute_error(pred_values, y),2)}')   # Средняя абсолютная ошибка регрессии

Веса признаков: [3657.8606821    81.43708049    5.62779018]
Bias: -14164.627426650812
Абсолютная ошибка предсказания: 52481.68


In [135]:
# Дополнительно создадим новый признак. Как он влияет на прогноз? Какова его прогностическая сила? 

data['mult'] = data['Number_of_passengers'] * data['Price']
data.head()

,Number_of_passengers,Age,Price,Amount,mult
0,2,61,22601,137144,45202
1,3,25,21077,191231,63231
2,4,28,26904,136058,107616
3,4,72,9861,40982,39444
4,2,15,3471,36248,6942


In [136]:
X = data[['mult']]
y = data['Amount']

new_model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.3, 
                                                        random_state=42 )
new_model.fit(X_train, y_train)

print(f'Веса признаков: {new_model.coef_}') 
print(f'Bias: {new_model.intercept_}') 
pred_values = new_model.predict(data[['mult']]) 
print(f'Абсолютная ошибка предсказания: {round(mean_absolute_error(pred_values, y),2)}')   
# Видим, что созданный признак дает еще бОльшую ошибку - он явно не годится для прогнозирования результатов 

Веса признаков: [1.53083316]
Bias: 18380.392576071827
Абсолютная ошибка предсказания: 55154.7


In [137]:
# Создадим ячейки NaN 
print(data['Amount'].replace([1538,10686],np.NaN).count())

815


In [126]:
data['Amount'] = data['Amount'].replace([1538,10686],np.NaN)
data.head()

,Number_of_passengers,Age,Price,Amount,mult
0,3,79,3860,32418.0,11580
1,3,67,8439,77489.0,25317
2,4,12,12115,110573.0,48460
3,3,16,23076,209222.0,69228
4,3,59,26178,NaN,78534


In [141]:
# Применим стратегию замены пустых значений столбца выставленного пассажирам счета на медианное значение данного признака:
data.loc[data[data['Amount'].isna()].index ,'Amount'] = data['Amount'].median() 

In [146]:
# data['Amount'].head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890 entries, 0 to 889
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Number_of_passengers  890 non-null    int32
 1   Age                   890 non-null    int32
 2   Price                 890 non-null    int32
 3   Amount                890 non-null    int32
 4   mult                  890 non-null    int32
dtypes: int32(5)
memory usage: 17.5 KB
